In [ ]:
from utils_new import *    

Load data from Zong et al. (2022), mouse 97045 session 20210307

Preprocess data, cluster neurons and visualize statistics

In [ ]:
xy_all = {}
movetimes_all = {}
spk_all = {}
coords_all = {}

data_dir = '/Users/erihe/OneDrive - NTNU/TDA_review/weijan'


mouse = '97045'
sessall = np.sort(glob.glob(data_dir + '/' + mouse + '/*'))
for sess in sessall[2:3]:
    print(sess)    
    NAT = h5py.File(sess + '/' + 'NAT.mat')
    nat_all = NAT[NAT['NAT'][()][0][0]][()]

    filtered_events = nat_all[np.arange(15,len(nat_all), 4),:]
    tt = nat_all[0,:]
    headpos = nat_all[1:3,:].T
    headdirection = nat_all[3,:]    
    speed = nat_all[4,:]

    sspikes = np.zeros(np.shape(filtered_events)).T
    for i in range(len(filtered_events)):
        if np.sum(np.isnan(filtered_events[i,:]))== len(tt):
            continue
        if np.isnan(filtered_events[i,0]):
            sspikes[1:-1,i] = scipy.interpolate.interp1d(tt[np.arange(1,len(tt),2)], filtered_events[i,np.arange(1,len(tt),2)])(tt[1:-1])
        else:
            sspikes[1:-1,i] = scipy.interpolate.interp1d(tt[np.arange(0,len(tt),2)], filtered_events[i,np.arange(0,len(tt),2)])(tt[1:-1])    
    sspikes = sspikes[1:-1,:]
    tt = tt[1:-1]
    headpos = headpos[1:-1,:]
    headdirection = headdirection[1:-1]
    speed = speed[1:-1]
    repremove = np.ones(len(sspikes[0,:]), dtype = bool)        
    try:
        NeuronInformation = sio.loadmat(sess + '/' + 'NeuronInformation.mat')['NeuronInformation']
        if len(NeuronInformation['RepeatCell'][()][0,0])>0:
            repremove[NeuronInformation['RepeatCell'][()][0,0][0].astype(int)-1] = False
    except:
        NeuronInformation = h5py.File(sess + '/' + 'NeuronInformation.mat')['NeuronInformation']
        if len(NeuronInformation['RepeatCell'][()])>0:
            repremove[NeuronInformation['RepeatCell'][()][0].astype(int)-1] = False
    del filtered_events, nat_all, NeuronInformation
    sspikes = sspikes[:, repremove]
    sspikes[np.isnan(sspikes)] = 0
    sspikes[sspikes<0.001] = 0
    spksum = np.mean(sspikes,0)
    indssort = np.where((spksum>0) & (spksum<10))[0]

    sig0  = 2
    sspk1 = sspikes[:,indssort]
    spknull0 = sspk1.sum(0)>0
    sspk1 = sspk1[:, spknull0 ]
    sspk1 = preprocessing.minmax_scale(sspk1, axis = 0)
    sspk1 = np.sqrt(gaussian_filter1d(sspk1, axis = 0, sigma = sig0))

    lenc = 10                
    lenspk, num_neurons = np.shape(sspk1)           
    Xcorr = cross_corr_dist(sspk1, lencorr = lenc)
    Xcorr[np.isnan(Xcorr)] = 1

    np.fill_diagonal(Xcorr,0)
    Xcorr1 = squareform(pdist(np.square(Xcorr), 'correlation'))

    thr = 0.85
    ind1 = get_ind(Xcorr1,thr, linkage = 'average', bPlot = True)
    bin_ind = np.bincount(ind1)
    numneuronsind = np.flip(np.argsort(bin_ind))


    movetimes0 = np.where((speed>5) & (np.sum(sspk1,1)>0))[0]

    num_neurons = len(sspk1[0,:])
    rmap = np.zeros((num_neurons, 25, 25))
    acorr = np.zeros((num_neurons, 25, 25))
    numbins1 = 25
    sig1 = 1

    for i in range(num_neurons):
        mtot_tmp, x_edge, y_edge,c2 = binned_statistic_2d(headpos[movetimes0,0], headpos[movetimes0,1],
                                  sspk1[movetimes0,i], statistic='mean', 
                                 bins=numbins1, range=None, expand_binnumbers=True)


        nans = np.isnan(mtot_tmp)
        mtot_tmp[nans] = np.mean(mtot_tmp[~nans])
        mtot_tmp = gaussian_filter(mtot_tmp,sigma = sig1)
        acorr[i, :,:] = pearson_correlate2d(mtot_tmp, mtot_tmp)
        rmap[i,:,:]  = mtot_tmp

    hd_info = np.zeros(num_neurons)
    for i in range(num_neurons):
        mtot, __, circ = binned_statistic(headdirection[movetimes0], sspk1[movetimes0,i], statistic = 'mean', bins = 30)
        mu = np.mean(sspk1[:,i])
        hd_info[i] = information_score_1d(mtot, circ-1, mu)

    scores = (('rmap', rmap), ('acorr2d', acorr), ('sum', np.sum(sspk1,0)), ('hd_info', hd_info))

    rel_inds = np.where((bin_ind>=30) & (bin_ind<=500))[0]                             
    for i in rel_inds:
        mod_ind1s = np.where(ind1==i)[0]
        if len(mod_ind1s)>=2:
            print('Mod ', i)
            print('num_neurons ', len(mod_ind1s))
            print(mod_ind1s)
            sspk2 = sspk1[:,mod_ind1s]
            sspk2 = preprocessing.scale(sspk2[movetimes0])

            scores_cluster(sspk2, scores,  mod_ind1s,headpos[movetimes0,0], headpos[movetimes0, 1], spk2 = [], 
                           num_example = 6, dim = min(10, len(mod_ind1s)), bUMAP = False)
            plt.show()


In [ ]:
fig = plt.figure(dpi = 300)
d2 = Xcorr1.copy()
d2 = d2[np.argsort(ind1), :]
d2 = d2[:,np.argsort(ind1)]    
plt.imshow(d2, 
           vmin = np.percentile(d2.flatten(),5),
           vmax = np.percentile(d2.flatten(),95),
          )
plt.axis('off')
bin_ind = np.bincount(ind1)
numneuronsind = np.flip(np.argsort(bin_ind))
print('num: ', bin_ind[numneuronsind[:10]])
print('ind: ', numneuronsind[:10])

plt.savefig('Figures/Zong_corrmat.png', transparent = True,  bbox_inches='tight', pad_inches=0.2)
plt.savefig('Figures/Zong_corrmat.pdf', transparent = True, bbox_inches='tight', pad_inches=0.2)

data = []
data_names = []
for i in range(len(d2)):
    data.append(pd.Series(d2[:,i]))
    data_names.extend(['col_' + str(i)])
df = pd.concat(data, ignore_index=True, axis=1)            
df.columns = data_names
df.to_excel("Source_data/ExtFig1c_crossmat.xlsx", sheet_name='crossmat')  


Fixate analysis on single cluster

In [ ]:
mod_inds = np.where(ind1==0)[0]
print(len(mod_inds))

sig0  = 2
sspk2 = sspikes[:,indssort]
spknull0 = sspk2.sum(0)>0
sspk2 = sspk2[:, spknull0 ]
sspk2 = sspk2[:, mod_inds]
sspk2 = preprocessing.minmax_scale(sspk2, axis = 0)

sspk2 = np.sqrt(gaussian_filter1d(sspk2, axis = 0, sigma = sig0))
movetimes0 = np.where((np.sum(sspk2,1)>0) & (speed>5))[0]
sspk2 = sspk2[movetimes0,:]
print(np.shape(sspk2))

Apply PCA to 6 dim and temporally downsample

In [ ]:
n_points = 1600
dim = 6
eps = 0.5
k = 1000

dim_red_spikes_move_scaled_bu, e1, e2, var_exp = pca(preprocessing.scale(sspk2,axis = 0), dim = dim)
dim_red_spikes_move_scaled_bu /= np.sqrt(e2[:dim])      

startindex = np.argmax(np.sum(np.abs(dim_red_spikes_move_scaled_bu),1))
movetimes1 = radial_downsampling(dim_red_spikes_move_scaled_bu, epsilon = eps, startindex = startindex)
indstemp = []
if n_points > len(movetimes1):
    n_points = len(movetimes1)
indstemp  = sample_denoising(dim_red_spikes_move_scaled_bu[movetimes1,:],  k, 
                                   n_points, 1, metric = 'cosine')[0]
indstemp = movetimes1[indstemp]


Compute the persistent homology of the downsampled neural ensemble activity

In [ ]:
print('n_points', n_points)
indstemp = indstemp[:n_points]
dim_red_spikes_move_scaled = dim_red_spikes_move_scaled_bu[indstemp,:]

d = squareform(pdist(dim_red_spikes_move_scaled[:,:], 'cosine'))
thresh = np.max(d[~np.isinf(d)])    
persistence = ripser(d, maxdim=1, coeff=47, do_cocycles= True, distance_matrix = True, thresh = thresh)   
print('movetimes1', len(movetimes1))
n_points = len(indstemp)

dgms = persistence['dgms']
fig = plot_barcode(dgms)



coords_ds, coords_ds_consistent = get_coords_consistent(persistence, coeff = 47, ph_classes = [0,1], bConsistent = True)
fig,ax = plt.subplots(1,1)
ax.plot(var_exp[:15], lw = 2.5)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
for i in range(len(coords_ds)):
    ax[i].plot(coords_ds[i,np.argsort(coords_ds[i,:])])
ax[2].scatter(*coords_ds[:2,:], s = 100)
for i in range(3):
    ax[i].set_aspect(1/ax[i].get_data_ratio())

fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
for i in range(len(coords_ds_consistent)):
    ax[i].plot(coords_ds_consistent[i,np.argsort(coords_ds_consistent[i,:])])
ax[2].scatter(*coords_ds_consistent[:2,:], s = 100)
for i in range(3):
    ax[i].set_aspect(1/ax[i].get_data_ratio())


In [ ]:

ff1 = ['Zong',]
for mouse_sess in ff1:
    print(mouse_sess)
    diagrams_roll = {}
    count = -1
    for i in range(0,101):
        if i == 0:
            f = np.load('Zong_dgms/dgms' + str(0) + '.npz', allow_pickle = True)
            dgms_real = f['dgms'][()][0]
            f.close()
            continue
        try :
            f = np.load('Zong_dgms/dgms' + str(i) + '.npz', allow_pickle = True)
            dgmstmp = f['dgms'][()]
            f.close()
            count += 1
            diagrams_roll[count] = dgmstmp[0]
        except:
            continue
    xmax = 1
    print(count)
    plot_barcode(dgms_real, diagrams_roll = diagrams_roll, percshuf = 99, dpi = 300, SaveSourceDataName = 'Zong_barcode')
    plt.savefig('Figures/Zong_barcode.png', transparent = True,  bbox_inches='tight', pad_inches=0.2)
    plt.savefig('Figures/Zong_barcode.pdf', transparent = True, bbox_inches='tight', pad_inches=0.2)    

In [ ]:
coords_mod1 = get_coords_all(sspk2, 
                             coords_ds_consistent,
                             np.arange(len(sspk2)),                             
                             indstemp,
                             dim = dim, 
                             bPCA = True,
                             bPred = False)


xx1 = headpos[movetimes0,0]
yy1 = headpos[movetimes0,1]
sig = 1
fig1, axs1 = plt.subplots(1,2, figsize = (6,4), dpi = 300)
fig2, axs2 = plt.subplots(1,2, figsize = (6,4), dpi = 300)
data = []
data_names = []
for c in [0,1,]:
    plt.hsv()
    axs1[c].scatter(xx1,yy1, c = coords_mod1[:,c], s = 5)
    axs1[c].set_aspect(1/axs1[c].get_data_ratio())
    axs1[c].axis('off')
    

    nans0 = ~np.isnan(coords_mod1[:,c])
    mtot, __, __, circ  = binned_statistic_2d(headpos[movetimes0,0][nans0],
                                              headpos[movetimes0,1][nans0],
                                              coords_mod1[:,c][nans0], 
                                              statistic = circmean, 
                                              bins = 50,
                                              expand_binnumbers = True)

    nans = np.isnan(mtot)
    sintot = np.sin(mtot)
    costot = np.cos(mtot)
    sintot[nans] = np.mean(sintot[~nans])
    costot[nans] = np.mean(costot[~nans])
    sintot = gaussian_filter(sintot,sig)
    costot = gaussian_filter(costot,sig)
    mtot = np.cos(np.arctan2(sintot, costot))
#            mtot = gaussian_filter(mtot, 1)

    plt.viridis()
    mtot[nans] = np.nan
    plt.viridis()
    axs2[c].imshow(mtot)
    axs2[c].axis('off')
    axs2[c].set_aspect(1/axs2[c].get_data_ratio())

    for i in range(len(mtot)):
        data.append(pd.Series(mtot[:,i]))
        data_names.extend(['circ' +str(c) + 'col' + str(i)])
    
df = pd.concat(data, ignore_index=True, axis=1)            
df.columns = data_names
df.to_excel('Source_data/Fig1c_Zong_spaceVtorus.xlsx', sheet_name='Zong_spaceVtorus')  
    
fig2.savefig('Figures/Zong_spaceVtorus.png', transparent = True, bbox_inches='tight', pad_inches=0.2)
fig2.savefig('Figures/Zong_spaceVtorus.pdf', transparent = True, bbox_inches='tight', pad_inches=0.2)    

In [ ]:
num_neurons = len(sspk2[0,:])

numfigs = 2
numw = 5
numh = int(np.ceil(num_neurons/numw))
outer1 = gridspec.GridSpec(1, numw)
fig = plt.figure(figsize=(np.ceil((numw*numfigs+numw-1)*1.05), np.ceil(numh*1.1)), dpi = 120)
plt.viridis()
nw = 0
sig1 = 1
numbins1 = 30

torsort = np.arange(num_neurons)#np.flip(np.argsort(pcorr))
cc = coords_mod1.copy()

for nn, n in enumerate(torsort):
    nnn = nn%numh
    if nnn == 0:
        outer2 = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec = outer1[nw], wspace = .3)
        gs2 = gridspec.GridSpecFromSubplotSpec(numh, numfigs, subplot_spec = outer2[0], hspace = 0.2,wspace = .0)
        nw += 1
    num_neurons = len(sspk2[0,:])

    ax = plt.subplot(gs2[nnn,0]) 
    mtot_tmp, x_edge, y_edge,c2 = binned_statistic_2d(xx1, yy1,
                                  sspk2[:,n], statistic='mean', 
                                 bins=numbins1, range=None, expand_binnumbers=True)


    nans = np.isnan(mtot_tmp)
    mtot_tmp[np.isnan(mtot_tmp)] = np.mean(mtot_tmp[~np.isnan(mtot_tmp)])
    nans = np.isnan(mtot_tmp)
    mtot_tmp[nans] = np.mean(mtot_tmp[~nans])
    mtot_tmp = gaussian_filter(mtot_tmp,sigma = sig1)
    minval = 0 
    maxval = np.percentile(mtot_tmp.flatten(), 97.5)
    mtot_tmp[nans] = -np.inf
    ax.imshow(mtot_tmp, origin = 'lower', vmin = minval, vmax = maxval)
    ax.set_xticks([])
    ax.set_yticks([])


    ax = plt.subplot(gs2[nnn,1]) 
    mtot_tmp, x_edge, y_edge,c2 = binned_statistic_2d(cc[:,0], cc[:,1],
                                          sspk2[:,n], statistic='mean', 
                                         bins=numbins1, range=None, expand_binnumbers=True)
    nans = np.isnan(mtot_tmp)
    mtot_tmp[np.isnan(mtot_tmp)] = np.mean(mtot_tmp[~np.isnan(mtot_tmp)])
    mtot_tmp = smooth_tuning_map(np.rot90(mtot_tmp,1), numbins1+1, sig1, bClose = True) 
    minval = 0 
    maxval = np.percentile(mtot_tmp.flatten(), 97.5)
    mtot_tmp[nans] = -np.inf
    ax.imshow(mtot_tmp, origin = 'lower', extent = [0,2*np.pi,0, 2*np.pi], 
              vmin = minval, vmax = maxval)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect(1/ax.get_data_ratio())
    r_box = transforms.Affine2D().skew_deg(15,15)
    for x in ax.images + ax.lines + ax.collections:
        trans = x.get_transform()
        x.set_transform(r_box+trans) 
        if isinstance(x, PathCollection):
            transoff = x.get_offset_transform()
            x._transOffset = r_box+transoff     
    ax.set_xlim(0, 2*np.pi + 3*np.pi/5)
    ax.set_ylim(0, 2*np.pi + 3*np.pi/5)
    ax.set_aspect('equal', 'box') 
    ax.axis('off')   
plt.show()


In [ ]:
num_neurons = len(sspk2[0,:])
torsort = np.arange(num_neurons)#np.flip(np.argsort(pcorr))

curr_inds = np.concatenate((np.argsort(torsort)[22:25], np.argsort(torsort)[6:9]))
num_neurons = len(curr_inds)

numfigs = 2
numw = 2
numh = int(np.ceil(num_neurons/numw))
outer1 = gridspec.GridSpec(1, numw)
fig = plt.figure(figsize=(np.ceil((numw*numfigs+numw-1)*1.05), np.ceil(numh*1.1)), dpi = 300)
plt.viridis()
nw = 0
sig1 = 1
numbins1 = 30

#torsort = np.arange(num_neurons)#np.flip(np.argsort(pcorr))
cc = coords_mod1.copy()

data = []
data_names = []
for nn, n in enumerate(curr_inds):
    nnn = nn%numh
    if nnn == 0:
        outer2 = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec = outer1[nw], wspace = .3)
        gs2 = gridspec.GridSpecFromSubplotSpec(numh, numfigs, subplot_spec = outer2[0], hspace = 0.2,wspace = .0)
        nw += 1
    num_neurons = len(sspk2[0,:])

    ax = plt.subplot(gs2[nnn,0]) 
    mtot_tmp, x_edge, y_edge,c2 = binned_statistic_2d(xx1, yy1,
                                  sspk2[:,n], statistic='mean', 
                                 bins=numbins1, range=None, expand_binnumbers=True)


    nans = np.isnan(mtot_tmp)
    mtot_tmp[np.isnan(mtot_tmp)] = np.mean(mtot_tmp[~np.isnan(mtot_tmp)])
    nans = np.isnan(mtot_tmp)
    mtot_tmp[nans] = np.mean(mtot_tmp[~nans])
    mtot_tmp = gaussian_filter(mtot_tmp,sigma = sig1)
    minval = 0 
    maxval = np.percentile(mtot_tmp.flatten(), 97.5)
    mtot_tmp[nans] = -np.inf
    ax.imshow(mtot_tmp, origin = 'lower', vmin = minval, vmax = maxval)
    ax.set_xticks([])
    ax.set_yticks([])
    
    for i in range(len(mtot_tmp)):
        data.append(pd.Series(mtot_tmp[:,i]))
        data_names.extend(['space' + str(i)])



    ax = plt.subplot(gs2[nnn,1]) 
    mtot_tmp, x_edge, y_edge,c2 = binned_statistic_2d(cc[:,0], cc[:,1],
                                          sspk2[:,n], statistic='mean', 
                                         bins=numbins1, range=None, expand_binnumbers=True)
    nans = np.isnan(mtot_tmp)
    mtot_tmp[np.isnan(mtot_tmp)] = np.mean(mtot_tmp[~np.isnan(mtot_tmp)])
    mtot_tmp = smooth_tuning_map(np.rot90(mtot_tmp,1), numbins1+1, sig1, bClose = True) 
    minval = 0 
    maxval = np.percentile(mtot_tmp.flatten(), 97.5)
    mtot_tmp[nans] = -np.inf
    ax.imshow(mtot_tmp, origin = 'lower', extent = [0,2*np.pi,0, 2*np.pi], 
              vmin = minval, vmax = maxval)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect(1/ax.get_data_ratio())
    r_box = transforms.Affine2D().skew_deg(15,15)
    for x in ax.images + ax.lines + ax.collections:
        trans = x.get_transform()
        x.set_transform(r_box+trans) 
        if isinstance(x, PathCollection):
            transoff = x.get_offset_transform()
            x._transOffset = r_box+transoff     
    ax.set_xlim(0, 2*np.pi + 3*np.pi/5)
    ax.set_ylim(0, 2*np.pi + 3*np.pi/5)
    ax.set_aspect('equal', 'box') 
    ax.axis('off')   

    for i in range(len(mtot_tmp)):
        data.append(pd.Series(mtot_tmp[:,i]))
        data_names.extend(['torus' + str(i)])
    
    df = pd.concat(data, ignore_index=True, axis=1)            
    df.columns = data_names
    df.to_excel('Source_data/ExtFig1c_Zong_single_tuning' + str(nn) + '.xlsx', sheet_name='Zong_single_tuning' + str(nn))  
plt.savefig('Figures/Zong_single_tuning.png', transparent = True, bbox_inches='tight', pad_inches=0.2)
plt.savefig('Figures/Zong_single_tuning.pdf', transparent = True, bbox_inches='tight', pad_inches=0.2)
